In [312]:
#import basic Libraries
import pandas as pd
import numpy as np
from tqdm import tqdm

#Visualizaiton imports
import seaborn as sns
import matplotlib.pyplot as plt

#Importing Sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Import Market Basket Models
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

%matplotlib inline

In [313]:
#import of all files from the data folder.
aisles = pd.read_csv('./data/aisles.csv')
departments = pd.read_csv('./data/departments.csv')
products = pd.read_csv('./data/products.csv')
order_products__prior = pd.read_csv('./data/order_products__prior.csv')
order_products__train = pd.read_csv('./data/order_products__train.csv')
orders = pd.read_csv('./data/orders.csv')
merged_products_aisles_departments = pd.read_csv('./data/merged_data.csv')
merged_products_aisles_departments = merged_products_aisles_departments.drop('Unnamed: 0', axis=1)

# Preprocessing

In [314]:
opt = order_products__train.merge(merged_products_aisles_departments, how='left', on='product_id')

In [315]:
opt.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,foodgroup
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,diary products and alternatives
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs,diary products and alternatives
2,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,vegetables and fruits
3,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,vegetables and fruits
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,others


In [316]:
#converting the id to string to preserve order.
opt['aisle_id']= opt['aisle_id'].astype(str)

In [317]:
#instanciate Oone hot encoder ot break out the data my aisle.
ohe = OneHotEncoder(handle_unknown='ignore')

In [318]:
#test to  see # of aisles
print('# of unique aisles =',len(opt['aisle_id'].unique()))
aisle = ohe.fit_transform(opt['aisle_id'].astype(str).values.reshape(-1,1))

# of unique aisles = 134


In [319]:
#one hot encode the aisles and create a dense matrix.
a_ohe = pd.DataFrame(aisle.toarray())
aisle = a_ohe.to_dense()

In [320]:
aisle.head()

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [321]:
#add order_id to the one hot encoded matrix
basket = opt[['order_id']].join(aisle)

In [322]:
basket.head()

,order_id,0,1,2,3,4,5,6,7,8,...,124,125,126,127,128,129,130,131,132,133
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [323]:
range(len(opt['aisle_id'].unique()))

range(0, 134)

In [324]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()

In [325]:
basket.head()

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [326]:
#covert any values to be a binary. 
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [327]:
basket_sets.head()

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [328]:
aisle_dict = dict(zip(opt['aisle_id'].unique().astype('Int64'),opt['aisle'].unique()))

/Users/soeunjang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """Entry point for launching an IPython kernel.


In [329]:
basket_sets.columns = list(range(1,len(aisle_dict)+1))
basket_sets.rename(columns=aisle_dict,inplace=True)

In [330]:
basket_sets.head()

,prepared soups salads,specialty cheeses,energy granola bars,instant foods,marinades meat preparation,other,packaged meat,bakery desserts,pasta sauce,kitchen supplies,...,trail mix snack mix,feminine care,body lotions soap,tortillas flat bread,frozen appetizers sides,hot cereal pancake mixes,dry pasta,beauty,muscles joints pain relief,specialty wines champagnes
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [331]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

Apriori is an algorithm for extracting frequent itemsets with applications in association rule learning. The apriori algorithm has been designed to operate on databases containing transactions, such as purchases by customers of a store. An itemset is considered as "frequent" if it meets a user-specified support threshold. For instance, if the support threshold is set to 0.5, a frequent itemset is defined as a set of items that occur together in at least 50% of all transactions in the database.

# Market Basket Results

In [332]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [333]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(fresh fruits),(fresh herbs),0.550099,0.096868,0.074446,0.135332,1.397072,0.021159,1.044484
1,(fresh herbs),(fresh fruits),0.096868,0.550099,0.074446,0.768529,1.397072,0.021159,1.943656
2,(fresh herbs),(fresh vegetables),0.096868,0.450975,0.083470,0.861684,1.910712,0.039785,3.969343
3,(fresh vegetables),(fresh herbs),0.450975,0.096868,0.083470,0.185088,1.910712,0.039785,1.108256
4,(fresh fruits),(packaged cheese),0.550099,0.237781,0.160911,0.292513,1.230178,0.030108,1.077361


Support is the relatvie frequency that the rules show up.
Confidence is a measure of the reliability of the rule.
Lift is the ratio of the observed support to that expected if the two rules were independet.

In [334]:
#sorts to see the lift greater than or equal to 1
rules[rules['lift'] >= 1.5].head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(fresh herbs),(fresh vegetables),0.096868,0.450975,0.083470,0.861684,1.910712,0.039785,3.969343
3,(fresh vegetables),(fresh herbs),0.450975,0.096868,0.083470,0.185088,1.910712,0.039785,1.108256
60,(yogurt),(milk),0.253405,0.223750,0.086991,0.343289,1.534251,0.030292,1.182026
61,(milk),(yogurt),0.223750,0.253405,0.086991,0.388787,1.534251,0.030292,1.221497
78,"(fresh fruits, packaged cheese)",(fresh vegetables),0.160911,0.450975,0.112538,0.699380,1.550816,0.039971,1.826306


# Market Basket for Products

In [335]:
reorders = opt[opt['reordered'] == 1]
#opt[opt['reordered'] == 1]

In [336]:
reorders['product_id'] = reorders['product_id'].astype('int64')

/Users/soeunjang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [337]:
# get list of hi volume products (products that occurr more than 1 time)
hivol = reorders.copy()['product_id'].value_counts().sort_values(ascending=False)\
    [reorders.copy()['product_id'].value_counts().sort_values(ascending=False) > 1].index.tolist()

In [338]:
# mask the reorders dataframe to only incluse dem hi
reorders = reorders[reorders['product_id'].isin(hivol)]

In [339]:
reorders['hi_dem'] = (reorders.copy()['product_id'].value_counts().sort_values(ascending=False)>1)

In [340]:
hidem_ord = reorders[reorders['hi_dem'] == True]

In [341]:
hidem_ord = hidem_ord.reset_index()

In [342]:
print('# of unique products =',len(hidem_ord['product_id'].unique()))
#tranposes the data to ensure that it is correctly fitted.
product = ohe.fit_transform(hidem_ord['product_id'].values.reshape(-1,1))

# of unique products = 4823


In [343]:
p_ohe = pd.DataFrame(product.toarray())
products = p_ohe.to_dense()

In [344]:
basket = hidem_ord[['order_id']].join(products)

In [345]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()

In [346]:
#covert any values to be a binary.
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [347]:
pro_dict = dict(zip(hidem_ord['product_id'].unique().astype('Int64'),hidem_ord['product_name'].unique()))

/Users/soeunjang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """Entry point for launching an IPython kernel.


In [348]:
pro_dict = dict(zip(list(range(0,len(hidem_ord['product_id'].unique()))),hidem_ord['product_name'].unique()))

In [349]:
#rename the columns for easier analysis in rules. 
basket_sets.columns = list(range(0,len(pro_dict)))
basket_sets.rename(columns=pro_dict,inplace=True)

In [350]:
basket_sets.sum()

Organic 4% Milk Fat Whole Milk Cottage Cheese                                                         4
Lightly Smoked Sardines in Olive Oil                                                                  1
Spring Water                                                                                          6
Cage Free Extra Large Grade AA Eggs                                                                   1
Roasted Turkey                                                                                        1
Organic Raspberries                                                                                   1
Organic Whole Strawberries                                                                            1
Natural Spring Water                                                                                  1
Whole Milk Greek Blended Vanilla Bean Yogurt                                                          5
Bag of Organic Bananas                                          

In [351]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.0005, use_colnames=True)

In [352]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
#rules = rules[['antecedants','consequents','confidence','lift']]

In [356]:
rules[(rules['lift'] >= 100) & (rules['confidence']>.5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
47,(Dha Omega 3 Reduced Fat 2% Milk),(Apple Pie),0.000799,0.002131,0.000533,0.666667,312.833333,0.000531,2.993607
82,"(Uncured, No Sugar, Hickory Smoked Bacon)",(Dairy Free Hazelnut Coconut Milk Creamer),0.000533,0.001865,0.000533,1.000000,536.285714,0.000532,inf
84,(Sonoma Traditional Flour Tortillas 10 Count),(Traditional Panettone),0.000799,0.005061,0.000533,0.666667,131.719298,0.000529,2.984816
99,(Restaurant Style Tortilla Rounds),(EnviroKidz Gluten Free & Wheat Free Gorilla M...,0.000799,0.003463,0.000533,0.666667,192.512821,0.000530,2.989611
102,(Organic Large Grade A Brown Eggs),(Diet Ginger Ale),0.000533,0.000799,0.000533,1.000000,1251.333333,0.000532,inf
103,(Diet Ginger Ale),(Organic Large Grade A Brown Eggs),0.000799,0.000533,0.000533,0.666667,1251.333333,0.000532,2.998402
150,(Olive Tapenade Hummus),"(Organic Nonfat Yogurt, Peach)",0.000533,0.009590,0.000533,1.000000,104.277778,0.000528,inf
236,(Original Citrus Sparkling Flavored Soda),(Organic Plain Yogurt),0.000799,0.003197,0.000533,0.666667,208.555556,0.000530,2.990410
242,(Organic White Corn & Wheat Tortillas),(Organic Low Fat Chocolate Milk),0.000533,0.006660,0.000533,1.000000,150.160000,0.000529,inf
246,(Royal Gala Apples),(Sweet Onions),0.000799,0.000533,0.000533,0.666667,1251.333333,0.000532,2.998402


In [353]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Honey Nut Chex Cereal),(Spring Water),0.033564,0.001598,0.000533,0.015873,9.931217,0.000479,1.014505
1,(Spring Water),(Honey Nut Chex Cereal),0.001598,0.033564,0.000533,0.333333,9.931217,0.000479,1.449654
2,(Light Sour Cream),(Organic Sliced Provalone Cheese),0.039158,0.001332,0.000533,0.013605,10.214966,0.000481,1.012443
3,(Organic Sliced Provalone Cheese),(Light Sour Cream),0.001332,0.039158,0.000533,0.400000,10.214966,0.000481,1.601403
4,"(Liquid Laundry Detergent, Original scent)",(Organic Sliced Provalone Cheese),0.008791,0.001332,0.000533,0.060606,45.503030,0.000521,1.063098
